# Helper Functions

In [ ]:
from starter import *
import random

def make_test():
    test = nx.empty_graph(16)
    for i in range(0, 3):
        test.add_edge(i, i+1, weight=100)
        test.add_edge(i+4, i+4+1, weight=100)
        test.add_edge(i+8, i+8+1, weight=100)
        test.add_edge(i+12, i+12+1, weight=100)
        test.add_edge(i*4, (i+1)*4, weight=100)
        test.add_edge((i*4)+1, ((i+1)*4) + 1, weight=100)
        test.add_edge((i*4)+2, ((i+1)*4) + 2, weight=100)
        test.add_edge((i*4)+3, ((i+1)*4) + 3, weight=100)
    return test
    
def new_first(G, curr, old_team, new_team, old_first):
    for z in G.neighbors(curr):
        if G.nodes[z]["team"] == old_team:
            old_first -= G[curr][z]["weight"]
        if G.nodes[z]["team"] == new_team:
            old_first += G[curr][z]["weight"]
    return old_first

def new_third(team_sizes, old_team, new_team, old_third, V, k):
    old_b = (math.log(old_third))/B_EXP
    b_i = (team_sizes[old_team]/V) - (1/k)
    b_j = (team_sizes[new_team]/V) - (1/k)
    rad = ((old_b**2) - (b_i**2) - (b_j**2) + (b_i - (1/V))**2 + (b_j + (1/V))**2)
    if rad < 0:
        rad = 0
    return math.exp(B_EXP*math.sqrt(rad))
    
def small(G: nx.Graph):
    # TODO add edges to complete the small input
    while G.size(weight = "weight") < 500_000:
        u = random.choice(range(0, N_SMALL))
        v = random.choice(list(range(0, u)) + list(range(u + 1, N_SMALL)))
        w = random.choice(range(1, 1_001))
        G.add_edge(u, v, weight=w)
    
def medium(G: nx.Graph):
    # TODO add edges to complete the medium input
    while G.size(weight = "weight") < 500_000:
        u = random.choice(range(0, N_MEDIUM))
        v = random.choice(list(range(0, u)) + list(range(u + 1, N_MEDIUM)))
        w = random.choice(range(1, 1_001))
        G.add_edge(u, v, weight=w)
    
def large(G: nx.Graph):
    # TODO add edges to complete the large input
    while G.size(weight = "weight") < 500_000:
        u = random.choice(range(0, N_LARGE))
        v = random.choice(list(range(0, u)) + list(range(u + 1, N_LARGE)))
        w = random.choice(range(1, 1_001))
        G.add_edge(u, v, weight=w)

def DFS_or_BFS(G: nx.Graph, k, sign):
    V = G.number_of_nodes()
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    team_sizes = {i:0 for i in range(1, k+1)}
    stack = [i[0] for i in nodes]
    visited = [False]*V
    while stack:
        biggest = stack[-1]
        queue = [biggest]
        while queue:
            curr = queue.pop(sign)
            if not visited[curr]:
                temp = team_sizes.copy()
                for neighbor in G.neighbors(curr):
                    if "team" in G.nodes[neighbor]:
                        adj_team = G.nodes[neighbor]["team"]
                        if adj_team in temp:
                            temp.pop(adj_team)
                if temp:
                    min_team = min(temp, key=temp.get)
                else:
                    edges = G.edges(curr, data=True)
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for i in edges:
                        neighbor = i[1]
                        if "team" in G.nodes[neighbor]:
                            adj_team = G.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                    min_team = min(adj_team_counter, key=adj_team_counter.get)
                G.nodes[curr]["team"] = min_team
                team_sizes[min_team] += 1
                stack.remove(curr)
                visited[curr] = True
                for neighbor in list(G.neighbors(curr))[::-1]:
                    if not visited[neighbor]:
                        queue.append(neighbor)
    old_first, old_second, old_third = score(G, separated=True)
    min_score = old_first + old_second + old_third
    for curr in range(V):
        curr_team = G.nodes[curr]["team"] 
        possible_teams = list(range(1, k+1))
        possible_teams.remove(curr_team)
        for neighbor in list(G.neighbors(curr)):
            adj_team = G.nodes[neighbor]["team"]
            if curr_team == adj_team: 
                if adj_team in possible_teams:
                    possible_teams.remove(adj_team)
        for new_team in possible_teams:
            temp_first = new_first(G, curr, curr_team, new_team, old_first)
            temp_third = new_third(team_sizes, curr_team, new_team, old_third, V, k)
            temp_score = temp_first + old_second + temp_third
            if temp_score < min_score:
                old_first = temp_first
                old_third = temp_third
                min_score = temp_score
                curr_team = new_team
        G.nodes[curr]["team"] = curr_team
    return

def partite(G, k):
    best_score = float("inf")
    best_G = G.copy()

    G_copy = G.copy()
    DFS_or_BFS(G_copy, k, 0)
    BFS_score = score(G_copy)
    if BFS_score < best_score:
        best_score = BFS_score
        best_G = G_copy.copy()
    G_copy = G.copy()
    DFS_or_BFS(G_copy, k, -1)
    DFS_score = score(G_copy)
    if DFS_score < best_score:
        best_score = DFS_score
        best_G = G_copy.copy()

    return best_G, best_score


# Dynamic Programming
(Attempted, Does not work)

In [ ]:
def solve_DP(G):
    V = G.number_of_nodes()
    upper_k = 2*int(((V+1)**(1/2)))
    possible_k = range(1, upper_k)
    memo = {}
    min_score = float("inf")
    min_G = G.copy()
    
    def DP_helper(G, nodes, k, best_score, G_copy_score_sep):
        if nodes == []:
            return (score(G, True)), G
        curr = nodes[0]
        best_G = G.copy()
        best_score_sep = G_copy_score_sep
        for new_team in range(1, k+1):
            G.nodes[curr]["team"] = new_team
            below_score_sep = DP_helper(G, nodes[1:], k, best_score, best_score_sep)[0]
            below_score = sum(below_score_sep)
            if below_score < best_score:
                best_score_sep = below_score_sep
                best_score = below_score
                best_G = G.copy()
        memo[G] = best_score_sep
        return best_score_sep, best_G

    for k in possible_k:
        G_copy, G_copy_score = partite(G.copy(), k)
        G_copy_score_sep = score(G_copy, True)
        team_sizes = {i:0 for i in range(1, k+1)}
        for i in G_copy:
            team_sizes[G_copy.nodes[i]["team"]] += 1
        k_score = sum(DP_helper(G_copy, list(G_copy.nodes), k, G_copy_score, G_copy_score_sep)[0])
        if k_score < min_score:
            min_score = k_score
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]["team"] = min_G.nodes[i]["team"]
    return

G = read_input('inputs/medium36.in')
solve_DP(G)
validate_output(G)
visualize(G)
score(G)

# Simulated Annealing
(Works but the runtime is too long. The results are not as good either.)

In [ ]:
import networkx as nx
import random
from starter import *
import math
import itertools

class SimulatedAnnealing:
    def __init__(self, initialSolution, solutionEvaluator, initialTemp, finalTemp, tempReduction, neighborOperator, iterationPerTemp=100, alpha=10, beta=5):
        self.solution = initialSolution
        self.evaluate = solutionEvaluator
        self.currTemp = initialTemp
        self.finalTemp = finalTemp
        self.iterationPerTemp = iterationPerTemp
        self.alpha = alpha
        self.beta = beta
        self.neighborOperator = neighborOperator

        if tempReduction == "linear":
            self.decrementRule = self.linearTempReduction
        else:
            print("Rule does not exist")
            self.decrementRule = None

    def linearTempReduction(self):
        self.currTemp -= self.alpha

    def isTerminationCriteriaMet(self):
        # can add more termination criteria
        return self.currTemp <= self.finalTemp

    def run(self):
        while not self.isTerminationCriteriaMet():
            # iterate that number of times
            for i in range(self.iterationPerTemp):
                # get all neighbors
                # newSolution, newScore = self.neighborOperator(self.solution)
                neighbors = self.neighborOperator(self.solution)
                # pick random neighbor
                newSolution = random.choice(neighbors)
                # get the cost between the two solutions
                old_graph, new_graph = self.solution, newSolution
                cost = self.evaluate(old_graph) - self.evaluate(new_graph)
                # if the new solution is better, accept it
                if cost >= 0:
                    self.solution = newSolution
                # if the new solution is not better, accept it with a probability of e^(-cost/temp)
                else:
                    try:
                        if random.uniform(0, 1) < math.exp(-cost / self.currTemp):
                            self.solution = newSolution
                    except:
                        pass
            # decrement the temperature
            self.decrementRule()

        return self.solution, score(self.solution)


def solve_sa_main(G: nx.Graph):
    """Simulated Annealing solution"""
    def bipartite(G: nx.Graph):
        G_copy = G.copy()
        nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
        stack = [i[0] for i in nodes]
        visited = [False]*G_copy.number_of_nodes()
        parent_team = [None]*G_copy.number_of_nodes()
        team_sizes = {1:0, 2:0}
        while stack:
            biggest = stack[-1]
            queue = [biggest]
            while queue:
                curr = queue.pop()
                if not visited[curr]:
                    min_team = min(team_sizes, key=team_sizes.get)
                    if parent_team[curr] == None:
                        G_copy.nodes[curr]["team"] = min_team
                        team_sizes[min_team] += 1
                    elif parent_team[curr] == 1:
                        G_copy.nodes[curr]["team"] = 2
                        team_sizes[2] += 1
                    elif parent_team[curr] == 2:
                        G_copy.nodes[curr]["team"] = 1
                        team_sizes[1] += 1
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
                    for i in edges:
                        neighbor = i[1]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                            parent_team[neighbor] = G_copy.nodes[curr]["team"]
                    stack.remove(curr)
                    visited[curr] = True
        return G_copy

    def initial_solution(G: nx.Graph, k):
        teams_nodes = {i: [] for i in range(1, k + 1)} # team_id: [nodes]
        teams_count = {i: 0 for i in range(1, k + 1)}  # team_id: count
        V = G.number_of_nodes()
        members_per_team = V // k # roughly equal size (floor div)

        # assign a random amount of penguins to random teams
        seen_nodes = set()
        assign_count = random.randint(1, V - (V//2))
        G_copy = G.copy()
        for node in G_copy:
            if assign_count > 0:
                id = random.randint(1, k)
                G_copy.nodes[node]['team'] = id
                teams_count[id] += 1
                teams_nodes[id].append(node)
                seen_nodes.add(node)
                assign_count -= 1
        V -= V//2

        # for remaining penguins, add that to team with least members up to members_per_team
        stop = 10
        while V > 0:
            if len(seen_nodes) < G_copy.number_of_nodes() and stop > 0:
                least_team = min(teams_count, key=teams_count.get)
                if teams_count[least_team] < members_per_team:
                    while teams_count[least_team] < members_per_team:
                        for node in G_copy:
                            if node not in seen_nodes:
                                G_copy.nodes[node]['team'] = least_team
                                teams_nodes[least_team].append(node)
                                teams_count[least_team] += 1
                                V -= 1
                else:
                    stop -= 1
            else:
                break
        
        if V > 0:
            print("Not all nodes were assigned")
            return None
        return G_copy

    def get_teams(G: nx.Graph):
        teams = {}
        for node in G.nodes:
            if not G.nodes[node]: # no team is originally assigned, there should be no team 0's though..
                teams[0] = [node]
            else:
                id = G.nodes[node]['team']
                if id not in teams:
                    teams[id] = [node]
                else:
                    teams[id].append(node)
        return teams
    
    def neighbors_operator(G: nx.Graph):
        def swap_once(G):
            # select two random penguins to swap
            # first, select two random teams
            teams = get_teams(G)
            teams_2 = random.sample(teams.keys(), 2)
            team0 = teams_2[0]
            team1 = teams_2[1]
            # then, select two random penguins from selected teams
            node0 = random.choice(teams[team0])
            node1 = random.choice(teams[team1])
            # perform swap
            G_copy = G.copy()
            G_copy.nodes[node1]['team'] = 0
            G_copy.nodes[node0]['team'] = team1
            G_copy.nodes[node1]['team'] = team0
            return G_copy

        # get all combinations
        teams = get_teams(G)
        node_teamid = {}
        for i in teams:
            for node in teams[i]:
                node_teamid[node] = i

        # select half of nodes to find combinations of
        nodes = random.sample([i for i in range(G.number_of_nodes())], G.number_of_nodes()//2)
        # swaps = list(itertools.combinations([node for node in G], 2))
        swaps = list(itertools.combinations(nodes, 2))
        # Filter out combinations where the two penguins are from the same team
        swaps_filtered = [swap for swap in swaps if node_teamid[swap[0]] != node_teamid[swap[1]]]
        #swap
        swapped_graphs = []
        if swapped_graphs == []: return [swap_once(G)] # in case there are no combinations, just swap once (should be unlikely though)
        for node0, node1 in swaps_filtered:
            G_copy = G.copy()
            team0 = G_copy.nodes[node0]['team']
            team1 = G_copy.nodes[node1]['team']
            G_copy.nodes[node1]['team'] = 0
            G_copy.nodes[node0]['team'] = team1
            G_copy.nodes[node1]['team'] = team0
            swapped_graphs.append(G_copy.copy())
        
        return swapped_graphs
        

    # alpha: between 0.5 and 0.999
    # starting temp: between 0.1 and 0.3
    # final temp: between 0.01 and 0.001
    
    initialTemp = 200
    finalTemp = 0.001
    tempReduction = "linear"
    iterationPerTemp = 10 * G.number_of_nodes() # arbritary number
    alpha = 0.95

    bip = bipartite(G)
    graphs = [(bip, score(bip))] # holds tuple (G:nx.Graph, score of G)
    for i in range(1): # run a few times for potentially better scores?
        for k in range(3, int(math.ceil(2 * math.sqrt(G.number_of_nodes())))):
            initialSolution = initial_solution(G, k)
            sa = SimulatedAnnealing(initialSolution=initialSolution, solutionEvaluator=score, initialTemp=initialTemp, finalTemp=finalTemp, tempReduction=tempReduction, neighborOperator=neighbors_operator, iterationPerTemp=iterationPerTemp, alpha=alpha)
            sol, sol_cost = sa.run()
            if sol not in graphs:
                graphs.append((sol ,sol_cost))

    # choose the best team size by looking at score
    res = min(graphs, key=lambda x: x[1])
    teams = get_teams(res[0])
    for id in teams:
        for node in teams[id]:
            G.nodes[node]['team'] = id
    return

# Solution #1 
Simple:  DFS -> Assign teams by smallest adjacent team

In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    possible_k = list(range(1, 2*int(V**(1/2))))
    min_cost = float('inf')
    min_G = G.copy()
    for k in possible_k:
        G_copy = G.copy()
        nodes = sorted(G_copy.degree(weight="weight"), key=lambda x: x[1])
        stack = [i[0] for i in nodes]

        while stack:
            curr = stack.pop()
            edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
            adj_team_counter = {i:0 for i in range(1, k+1)}
            for i in edges:
                neighbor = i[1]
                if "team" in G_copy.nodes[neighbor]:
                    adj_team_counter[G_copy.nodes[neighbor]["team"]] += i[2]["weight"]
            smallest_team = min(adj_team_counter, key=adj_team_counter.get)
            G_copy.nodes[curr]["team"] = smallest_team

        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


# Solution #2
Combined: DFS -> Assign all nodes to smallest adjacent team -> Loop to choose between min_adj_team and min_total_team

In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float('inf')
    min_G = G.copy()
    possible_k = list(range(1, V+1)) # OPTIMIZE: what upper-bound to use for k?
    for k in possible_k:
        G_copy = G.copy()
        nodes = sorted(G_copy.degree(weight="weight"), key=lambda x: x[1])
        stack = [i[0] for i in nodes]
        team_sizes = {i:0 for i in range(1, k+1)}
        # Assign every node to the min neighbor team (preprocess the graph)
        while stack:
            curr = stack.pop()
            edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
            adj_team_counter = {i:0 for i in range(1, k+1)}
            for i in edges:
                neighbor = i[1]
                if "team" in G_copy.nodes[neighbor]:
                    adj_team_counter[G_copy.nodes[neighbor]["team"]] += i[2]["weight"]
            smallest_team = min(adj_team_counter, key=adj_team_counter.get)
            G_copy.nodes[curr]["team"] = smallest_team

        # Choose between min total team and min adjacent team
        stack = [i[0] for i in nodes]
        while stack:
            curr = stack.pop()
            edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
            adj_team_counter = {i:0 for i in range(1, k+1)}
            for i in edges:
                neighbor = i[1]
                adj_team = G_copy.nodes[neighbor]["team"]
                adj_team_counter[adj_team] += i[2]["weight"]
            smallest_adj_team = min(adj_team_counter, key=adj_team_counter.get)
            smallest_total_team = min(team_sizes, key=team_sizes.get)

            G_copy.nodes[curr]["team"] = smallest_adj_team
            adj_score = score(G_copy)
            G_copy.nodes[curr]["team"] = smallest_total_team
            tot_score = score(G_copy)
            
            if smallest_adj_team != smallest_total_team and adj_score < tot_score:
                G_copy.nodes[curr]["team"] = smallest_adj_team
                team_sizes[smallest_adj_team] += 1
                curr_cost = adj_score
            else:
                G_copy.nodes[curr]["team"] = smallest_total_team
                team_sizes[smallest_total_team] += 1
                curr_cost = tot_score

        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
        else:
            break
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


# Solution #3
Combined with constant-time score update


In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    upper_k = 2*int(((V+1)**(1/2)))
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = list(range(1, upper_k))
    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        nodes = sorted(G_copy.degree(weight="weight"), key=lambda x: x[1])

        stack = [i[0] for i in nodes]
        while stack:
            curr = stack.pop()
            edges = G_copy.edges(curr, data=True)
            adj_team_counter = {i:0 for i in range(1, k+1)}
            for i in edges:
                neighbor = i[1]
                if "team" in G_copy.nodes[neighbor]:
                    adj_team = G_copy.nodes[neighbor]["team"]
                    adj_team_counter[adj_team] += i[2]["weight"]
            smallest_team = min(adj_team_counter, key=adj_team_counter.get)
            G_copy.nodes[curr]["team"] = smallest_team
            team_sizes[smallest_team] += 1
        
        curr_first, curr_second, curr_third = score(G_copy, separated=True)
        curr_cost = curr_first + curr_second + curr_third
        stack = [i[0] for i in nodes]
        while stack:
            curr = stack.pop()
            edges = G_copy.edges(curr, data=True)
            adj_team_counter = {i:0 for i in range(1, k+1)}
            for i in edges:
                neighbor = i[1]
                if "team" in G_copy.nodes[neighbor]:
                    adj_team = G_copy.nodes[neighbor]["team"]
                    adj_team_counter[adj_team] += i[2]["weight"]
            
            old_team = G_copy.nodes[curr]["team"]
            smallest_adj_team = min(adj_team_counter, key=adj_team_counter.get)
            smallest_total_team = min(team_sizes, key=team_sizes.get)
            
            adj_first = new_first(G_copy, curr, old_team, smallest_adj_team, curr_first)
            adj_third = new_third(team_sizes, old_team, smallest_adj_team, curr_third, V, k)
            adj_score = adj_first + curr_second + adj_third

            tot_first = new_first(G_copy, curr, old_team, smallest_total_team, curr_first)
            tot_third = new_third(team_sizes, old_team, smallest_total_team, curr_third, V, k)
            tot_score = tot_first + curr_second + tot_third

            if adj_score < curr_cost:
                G_copy.nodes[curr]["team"] = smallest_adj_team
                team_sizes[smallest_adj_team] += 1
                team_sizes[old_team] -= 1
                curr_first = adj_first
                curr_third = adj_third
                curr_cost = adj_score
                old_team = G_copy.nodes[curr]["team"]
            if tot_score < curr_cost:
                G_copy.nodes[curr]["team"] = smallest_total_team
                team_sizes[smallest_total_team] += 1
                team_sizes[old_team] -= 1
                curr_first = tot_first
                curr_third = tot_third
                curr_cost = tot_score
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


# Solution #4
DFS -> Assign all nodes to smallest total team -> DFS -> Choose between min_adj_team and min_total_team

In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, 2*int(((V+1)**(1/2))))
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    
    for k in possible_k:
        G_copy = G.copy()
        curr_G, curr_score = partite(G_copy, k)
        if curr_score < min_cost:
            min_cost = curr_score
            min_G = curr_G.copy()
            
    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}

        visited = [False]*V
        stack = [i[0] for i in nodes]
        while stack:
            biggest_node = stack[-1]
            queue = [biggest_node]
            while queue:
                curr = queue.pop()
                if not visited[curr]:
                    min_team = min(team_sizes, key=team_sizes.get)
                    G_copy.nodes[curr]["team"] = min_team
                    team_sizes[min_team] += 1
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
                    for i in edges:
                        neighbor = i[1]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    stack.remove(curr)
                    visited[curr] = True
        curr_first, curr_second, curr_third = score(G_copy, separated=True)
        curr_score = curr_first + curr_second + curr_third

        visited = [False]*V
        stack = [i[0] for i in nodes]
        while stack:
            biggest_node = stack[-1]          
            queue = [biggest_node]
            while queue:
                curr = queue.pop()
                if not visited[curr]:
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for neighbor in list(G_copy.neighbors(curr))[::-1]:
                        if "team" in G_copy.nodes[neighbor]:
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    
                    old_team = G_copy.nodes[curr]["team"]
                    smallest_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                    smallest_total_team = min(team_sizes, key=team_sizes.get)
                    
                    adj_first = new_first(G_copy, curr, old_team, smallest_adj_team, curr_first)
                    adj_third = new_third(team_sizes, old_team, smallest_adj_team, curr_third, V, k)
                    adj_score = adj_first + curr_second + adj_third

                    tot_first = new_first(G_copy, curr, old_team, smallest_total_team, curr_first)
                    tot_third = new_third(team_sizes, old_team, smallest_total_team, curr_third, V, k)
                    tot_score = tot_first + curr_second + tot_third
                    
                    if tot_score < curr_score:
                        G_copy.nodes[curr]["team"] = smallest_total_team
                        team_sizes[smallest_total_team] += 1
                        team_sizes[old_team] -= 1
                        curr_first = tot_first
                        curr_third = tot_third
                        curr_score = tot_score
                    old_team = G_copy.nodes[curr]["team"]   
                    if adj_score < curr_score:
                        G_copy.nodes[curr]["team"] = smallest_adj_team
                        team_sizes[smallest_adj_team] += 1
                        team_sizes[old_team] -= 1
                        curr_first = adj_first
                        curr_third = adj_third
                        curr_score = adj_score
                    stack.remove(curr)
                    visited[curr] = True
            
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


# Solution #5
Solution #4 but repeat second while loop k times

In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    upper_k = 2*int(((V+1)**(1/2)))
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, upper_k)
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = nodes[-1][0]
    
    for k in possible_k:
        G_copy = G.copy()
        curr_G, curr_score = partite(G_copy, k)
        if curr_score < min_cost:
            min_cost = curr_score
            min_G = curr_G.copy()
    
    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        visited = [False]*V
        stack = [i[0] for i in nodes]
        while stack:
            biggest_node = stack[-1]
            queue = [biggest_node]
            while queue:
                curr = queue.pop()
                # old_team = G_copy.nodes[curr]["team"]
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                    for i in edges:
                        neighbor = i[1]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    min_team = min(team_sizes, key=team_sizes.get)
                    G_copy.nodes[curr]["team"] = min_team
                    team_sizes[min_team] += 1
                    # team_sizes[old_team] -= 1
                    stack.remove(curr)
                    visited[curr] = True
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            visited = [False]*V
            stack = [i[0] for i in nodes]
            while stack:
                biggest_node = stack[-1]          
                queue = [biggest_node]
                while queue:
                    curr = queue.pop()
                    if not visited[curr]:
                        adj_team_counter = {i:0 for i in range(1, k+1)}
                        for neighbor in list(G_copy.neighbors(curr))[::-1]:
                            if "team" in G_copy.nodes[neighbor]:
                                adj_team = G_copy.nodes[neighbor]["team"]
                                adj_team_counter[adj_team] += i[2]["weight"]
                            if not visited[neighbor]:
                                queue.append(neighbor)
                        
                        old_team = G_copy.nodes[curr]["team"]
                        smallest_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                        smallest_total_team = min(team_sizes, key=team_sizes.get)
                        
                        adj_first = new_first(G_copy, curr, old_team, smallest_adj_team, curr_first)
                        adj_third = new_third(team_sizes, old_team, smallest_adj_team, curr_third, V, k)
                        adj_score = adj_first + curr_second + adj_third

                        tot_first = new_first(G_copy, curr, old_team, smallest_total_team, curr_first)
                        tot_third = new_third(team_sizes, old_team, smallest_total_team, curr_third, V, k)
                        tot_score = tot_first + curr_second + tot_third
                        
                        if adj_score < curr_score:
                            G_copy.nodes[curr]["team"] = smallest_adj_team
                            team_sizes[smallest_adj_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = adj_first
                            curr_third = adj_third
                            curr_score = adj_score
                        old_team = G_copy.nodes[curr]["team"]   
                        if tot_score < curr_score:
                            G_copy.nodes[curr]["team"] = smallest_total_team
                            team_sizes[smallest_total_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = tot_first
                            curr_third = tot_third
                            curr_score = tot_score
                        stack.remove(curr)
                        visited[curr] = True
                    
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


# Solution #6: 4 Modular Parts

In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    upper_k = 2*int(((V+1)**(1/2)))
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, upper_k)
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = sorted(G.degree(weight="weight"), key=lambda x: x[1])[-1][0]
    
    for k in possible_k:
        G_copy = G.copy()
        curr_G, curr_score = partite(G_copy, k)
        if curr_score < min_cost:
            min_cost = curr_score
            min_G = curr_G.copy()

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        '''
        DFS TO PREPROCESS ALL G_COPY NDOES TO MIN_TOT_TEAM
        '''
        visited = [False]*V
        stack = [i[0] for i in nodes]
        while stack:
            biggest_node = stack[-1]
            queue = [biggest_node]
            while queue:
                curr = queue.pop()
                # old_team = G_copy.nodes[curr]["team"]
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1), reverse=True)
                    for i in edges:
                        neighbor = i[1]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    min_team = min(team_sizes, key=team_sizes.get)
                    G_copy.nodes[curr]["team"] = min_team
                    team_sizes[min_team] += 1
                    # team_sizes[old_team] -= 1
                    stack.remove(curr)
                    visited[curr] = True
        '''
        DFS TO PREPROCESS ALL G_COPY NODES TO MIN_ADJ_TEAM 
        '''
        visited = [False]*V
        stack = [i[0] for i in nodes]
        while stack:
            biggest_node = stack[-1]
            queue = [biggest_node]
            while queue:
                curr = queue.pop()
                old_team = G_copy.nodes[curr]["team"]
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for i in edges:
                        neighbor = i[1]
                        if "team" in G_copy.nodes[neighbor]:
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    smallest_team = min(adj_team_counter, key=adj_team_counter.get)
                    G_copy.nodes[curr]["team"] = smallest_team
                    team_sizes[smallest_team] += 1
                    team_sizes[old_team] -= 1
                    stack.remove(curr)
                    visited[curr] = True     
        '''
        ITERATE THROUGH SORTED NODES AND CHOOSE BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            queue = [i[0] for i in nodes][::-1]
            for curr in queue:
                edges = G_copy.edges(curr, data=True)
                adj_team_counter = {i:0 for i in range(1, k+1)}
                for i in edges:
                    neighbor = i[1]
                    adj_team = G_copy.nodes[neighbor]["team"]
                    adj_team_counter[adj_team] += i[2]["weight"]
                old_team = G_copy.nodes[curr]["team"]
                min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                min_tot_team = min(team_sizes, key=team_sizes.get)
                
                adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                adj_score = adj_first + curr_second + adj_third

                tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                tot_score = tot_first + curr_second + tot_third

                if adj_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_adj_team
                    team_sizes[min_adj_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = adj_first
                    curr_third = adj_third
                    curr_score = adj_score 
                old_team = G_copy.nodes[curr]["team"] 
                if tot_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_tot_team
                    team_sizes[min_tot_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = tot_first
                    curr_third = tot_third
                    curr_score = tot_score
        '''
        DFS AND ASSIGN NODES BY CHOOSING BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            visited = [False]*V
            stack = [i[0] for i in nodes]
            while stack:
                biggest_node = stack[-1]
                queue = [biggest_node]
                while queue:
                    curr = queue.pop()
                    old_team = G_copy.nodes[curr]["team"]
                    if not visited[curr]:
                        edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                        adj_team_counter = {i:0 for i in range(1, k+1)}
                        for i in edges:
                            neighbor = i[1]
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                            if not visited[neighbor]:
                                queue.append(neighbor)
                        min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                        min_tot_team = min(team_sizes, key=team_sizes.get)
                        
                        adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                        adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                        adj_score = adj_first + curr_second + adj_third

                        tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                        tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                        tot_score = tot_first + curr_second + tot_third

                        if adj_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_adj_team
                            team_sizes[min_adj_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = adj_first
                            curr_third = adj_third
                            curr_score = adj_score 
                        old_team = G_copy.nodes[curr]["team"]
                        if tot_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_tot_team
                            team_sizes[min_tot_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = tot_first
                            curr_third = tot_third
                            curr_score = tot_score
                        stack.remove(curr)
                        visited[curr] = True   
                    
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
        else:
            break
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


# Other Solutions

In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, 2*int(((V+1)**(1/2))))
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = nodes[-1][0]

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        '''
        DFS TO PREPROCESS ALL G_COPY NDOES TO MIN_TOT_TEAM
        '''
        visited = [False]*V
        stack = [i[0] for i in nodes]
        for curr in range(V):
            if not [v for v in G_copy.neighbors(curr)]:
                min_team = min(team_sizes, key=team_sizes.get)
                G_copy.nodes[curr]["team"] = min_team
                team_sizes[min_team] += 1
                stack.remove(curr)
                visited[curr] = True
        while stack:
            biggest_node = stack[-1]
            queue = [biggest_node]
            while queue:
                curr = queue.pop(0)
                # old_team = G_copy.nodes[curr]["team"]
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for i in edges:
                        neighbor = i[1]
                        if "team" in G_copy.nodes[neighbor]:
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                    smallest_team = min(adj_team_counter, key=adj_team_counter.get)
                    G_copy.nodes[curr]["team"] = smallest_team
                    team_sizes[smallest_team] += 1
                    # team_sizes[old_team] -= 1
                    stack.remove(curr)
                    visited[curr] = True     
        '''
        ITERATE THROUGH SORTED NODES AND CHOOSE BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            queue = [i[0] for i in nodes]
            visited = [False for _ in range(V)]
            while queue:
                curr = queue.pop()
                edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                adj_team_counter = {i:0 for i in range(1, k+1)}
                for i in edges:
                    neighbor = i[1]
                    if "team" in G_copy.nodes[neighbor]:
                        adj_team = G_copy.nodes[neighbor]["team"]
                        adj_team_counter[adj_team] += i[2]["weight"]
                
                old_team = G_copy.nodes[curr]["team"]
                min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                min_tot_team = min(team_sizes, key=team_sizes.get)
                
                adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                adj_score = adj_first + curr_second + adj_third

                tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                tot_score = tot_first + curr_second + tot_third

                if adj_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_adj_team
                    team_sizes[min_adj_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = adj_first
                    curr_third = adj_third
                    curr_score = adj_score
                if tot_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_tot_team
                    team_sizes[min_tot_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = tot_first
                    curr_third = tot_third
                    curr_score = tot_score
        '''
        DFS AND ASSIGN NODES BY CHOOSING BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            visited = [False]*V
            stack = [i[0] for i in nodes]
            while stack:
                biggest_node = stack[-1]
                queue = [biggest_node]
                while queue:
                    curr = queue.pop(0)
                    old_team = G_copy.nodes[curr]["team"]
                    if not visited[curr]:
                        edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                        adj_team_counter = {i:0 for i in range(1, k+1)}
                        for i in edges:
                            neighbor = i[1]
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                            if not visited[neighbor]:
                                queue.append(neighbor)
                        min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                        min_tot_team = min(team_sizes, key=team_sizes.get)
                        
                        adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                        adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                        adj_score = adj_first + curr_second + adj_third

                        tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                        tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                        tot_score = tot_first + curr_second + tot_third

                        if adj_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_adj_team
                            team_sizes[min_adj_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = adj_first
                            curr_third = adj_third
                            curr_score = adj_score 
                        old_team = G_copy.nodes[curr]["team"]
                        if tot_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_tot_team
                            team_sizes[min_tot_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = tot_first
                            curr_third = tot_third
                            curr_score = tot_score
                        
                        stack.remove(curr)
                        visited[curr] = True   
                    
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, 2*int(((V+1)**(1/2))))
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = sorted(G.degree(weight="weight"), key=lambda x: x[1])[-1][0]

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        '''
        RANDOMLY ASSIGN TEAMS TO NODES
        '''
        for i in range(V):
            team = random.choice(range(1, k+1))
            G_copy.nodes[i]["team"] = team
            team_sizes[team] += 1
        '''
        ITERATE THROUGH SORTED NODES AND ASSIGN TEAMS TO MIN_TOT_TEAM
        '''
        queue = [i[0] for i in nodes]
        for curr in queue:
            old_team = G_copy.nodes[curr]["team"]
            edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
            adj_team_counter = {i:0 for i in range(1, k+1)}
            for i in edges:
                neighbor = i[1]
                if "team" in G_copy.nodes[neighbor]:
                    adj_team = G_copy.nodes[neighbor]["team"]
                    adj_team_counter[adj_team] += i[2]["weight"]
            smallest_team = min(adj_team_counter, key=adj_team_counter.get)
            G_copy.nodes[curr]["team"] = smallest_team
            team_sizes[smallest_team] += 1
        '''
        DFS AND ASSIGN NODES BY CHOOSING BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            visited = [False]*V
            stack = [i[0] for i in nodes]
            while stack:
                biggest_node = stack[-1]
                queue = [biggest_node]
                while queue:
                    curr = queue.pop()
                    old_team = G_copy.nodes[curr]["team"]
                    if not visited[curr]:
                        edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                        adj_team_counter = {i:0 for i in range(1, k+1)}
                        for i in edges:
                            neighbor = i[1]
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                            if not visited[neighbor]:
                                queue.append(neighbor)
                        min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                        min_tot_team = min(team_sizes, key=team_sizes.get)
                        
                        adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                        adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                        adj_score = adj_first + curr_second + adj_third

                        tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                        tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                        tot_score = tot_first + curr_second + tot_third

                        if adj_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_adj_team
                            team_sizes[min_adj_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = adj_first
                            curr_third = adj_third
                            curr_score = adj_score 
                        old_team = G_copy.nodes[curr]["team"]
                        if tot_score < curr_score:
                            G_copy.nodes[curr]["team"] = min_tot_team
                            team_sizes[min_tot_team] += 1
                            team_sizes[old_team] -= 1
                            curr_first = tot_first
                            curr_third = tot_third
                            curr_score = tot_score
                        stack.remove(curr)
                        visited[curr] = True
                      
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, 2*int(((V+1)**(1/2))))
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        for i in range(V):
            team = random.choice(range(1, k+1))
            G_copy.nodes[i]["team"] = team
            team_sizes[team] += 1
        '''
        ITERATE THROUGH SORTED NODES AND ASSIGN TEAMS TO MIN_TOT_TEAM
        '''
        queue = [i[0] for i in nodes]
        while queue:
            curr = queue.pop()
            # old_team = G_copy.nodes[curr]["team"]
            min_team = min(team_sizes, key=team_sizes.get)
            G_copy.nodes[curr]["team"] = min_team
            team_sizes[min_team] += 1
            # team_sizes[old_team] -= 1
        '''
        ITERATE THROUGH SORTED NODES AND CHOOSE BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM 
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            queue = [i[0] for i in nodes]
            while queue:
                curr = queue.pop()
                edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                adj_team_counter = {i:0 for i in range(1, k+1)}
                for i in edges[::-1]:
                    neighbor = i[1]
                    if "team" in G_copy.nodes[neighbor]:
                        adj_team = G_copy.nodes[neighbor]["team"]
                        adj_team_counter[adj_team] += i[2]["weight"]
                
                old_team = G_copy.nodes[curr]["team"]
                min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                min_tot_team = min(team_sizes, key=team_sizes.get)
                
                adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                adj_score = adj_first + curr_second + adj_third

                tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                tot_score = tot_first + curr_second + tot_third

                if adj_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_adj_team
                    team_sizes[min_adj_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = adj_first
                    curr_third = adj_third
                    curr_score = adj_score
                elif tot_score < curr_score:
                    G_copy.nodes[curr]["team"] = min_tot_team
                    team_sizes[min_tot_team] += 1
                    team_sizes[old_team] -= 1
                    curr_first = tot_first
                    curr_third = tot_third
                    curr_score = tot_score
                    
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, 2*int(((V+1)**(1/2))))
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = nodes[-1][0]

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        for i in range(V):
            team = random.choice(range(1, k+1))
            G_copy.nodes[i]["team"] = team
            team_sizes[team] += 1
        '''
        ITERATE THROUGH SORTED NODES AND ASSIGN TEAMS TO MIN_ADJ_TEAM
        '''
        queue = [biggest_node]
        visited = [False for _ in range(V)]
        while queue:
            curr = queue.pop()
            if not visited[curr]:
                edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                adj_team_counter = {i:0 for i in range(1, k+1)}
                for i in edges:
                    neighbor = i[1]
                    if "team" in G_copy.nodes[neighbor]:
                        adj_team = G_copy.nodes[neighbor]["team"]
                        adj_team_counter[adj_team] += i[2]["weight"]
                    if not visited[neighbor]:
                        queue.append(neighbor)
                smallest_team = min(adj_team_counter, key=adj_team_counter.get)
                G_copy.nodes[curr]["team"] = smallest_team
                team_sizes[smallest_team] += 1
                visited[curr] = True
        '''
        DFS AND ASSIGN NODES BY CHOOSING BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            queue = [biggest_node]
            visited = [False for _ in range(V)]
            while queue:
                curr = queue.pop()
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for i in edges:
                        neighbor = i[1]
                        if "team" in G_copy.nodes[neighbor]:
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                        if not visited[neighbor]:
                            queue.append(neighbor)

                    old_team = G_copy.nodes[curr]["team"]
                    min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                    min_tot_team = min(team_sizes, key=team_sizes.get)
                    
                    adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                    adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                    adj_score = adj_first + curr_second + adj_third

                    tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                    tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                    tot_score = tot_first + curr_second + tot_third

                    if adj_score < curr_score:
                        G_copy.nodes[curr]["team"] = min_adj_team
                        team_sizes[min_adj_team] += 1
                        team_sizes[old_team] -= 1
                        curr_first = adj_first
                        curr_third = adj_third
                        curr_score = adj_score
                    if tot_score < curr_score:
                        G_copy.nodes[curr]["team"] = min_tot_team
                        team_sizes[min_tot_team] += 1
                        team_sizes[old_team] -= 1
                        curr_first = tot_first
                        curr_third = tot_third
                        curr_score = tot_score
                    visited[curr] = True
                    
        curr_cost = curr_score
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    min_cost = float("inf")
    min_G = G.copy()
    possible_k = range(1, 2*int(((V+1)**(1/2))))
    nodes = sorted(G.degree(weight="weight"), key=lambda x: x[1])
    biggest_node = nodes[-1][0]

    for k in possible_k:
        G_copy = G.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        '''
        ITERATE THROUGH SORTED NODES AND ASSIGN TEAMS TO MIN_TOT_TEAM
        '''
        queue = [biggest_node]
        visited = [False for _ in range(V)]
        while queue:
            curr = queue.pop()
            if not visited[curr]:
                edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                adj_team_counter = {i:0 for i in range(1, k+1)}
                for i in edges:
                    neighbor = i[1]
                    if "team" in G_copy.nodes[neighbor]:
                        adj_team = G_copy.nodes[neighbor]["team"]
                        adj_team_counter[adj_team] += i[2]["weight"]
                    if not visited[neighbor]:
                        queue.append(neighbor)
                smallest_team = min(adj_team_counter, key=adj_team_counter.get)
                G_copy.nodes[curr]["team"] = smallest_team
                team_sizes[smallest_team] += 1
                visited[curr] = True
        for i in range(V):
            if "team" not in G_copy.nodes[i]:
                min_team = min(team_sizes, key=team_sizes.get)
                G_copy.nodes[i]["team"] = min_team
                team_sizes[min_team] += 1
        '''
        DFS AND ASSIGN NODES BY CHOOSING BETWEEN MIN_ADJ_TEAM AND MIN_TOT_TEAM
        '''
        for _ in range(k):
            curr_first, curr_second, curr_third = score(G_copy, separated=True)
            curr_score = curr_first + curr_second + curr_third
            queue = [biggest_node]
            visited = [False for _ in range(V)]
            while queue:
                curr = queue.pop()
                if not visited[curr]:
                    edges = sorted(G_copy.edges(curr, data=True), key=lambda t: t[2].get('weight', 1))
                    adj_team_counter = {i:0 for i in range(1, k+1)}
                    for i in edges:
                        neighbor = i[1]
                        if "team" in G_copy.nodes[neighbor]:
                            adj_team = G_copy.nodes[neighbor]["team"]
                            adj_team_counter[adj_team] += i[2]["weight"]
                        if not visited[neighbor]:
                            queue.append(neighbor)
                            
                    old_team = G_copy.nodes[curr]["team"]
                    min_adj_team = min(adj_team_counter, key=adj_team_counter.get)
                    min_tot_team = min(team_sizes, key=team_sizes.get)
                    
                    adj_first = new_first(G_copy, curr, old_team, min_adj_team, curr_first)
                    adj_third = new_third(team_sizes, old_team, min_adj_team, curr_third, V, k)
                    adj_score = adj_first + curr_second + adj_third

                    tot_first = new_first(G_copy, curr, old_team, min_tot_team, curr_first)
                    tot_third = new_third(team_sizes, old_team, min_tot_team, curr_third, V, k)
                    tot_score = tot_first + curr_second + tot_third

                    if adj_score < curr_score:
                        G_copy.nodes[curr]["team"] = min_adj_team
                        team_sizes[min_adj_team] += 1
                        team_sizes[old_team] -= 1
                        curr_first = adj_first
                        curr_third = adj_third
                        curr_score = adj_score
                    if tot_score < curr_score:
                        G_copy.nodes[curr]["team"] = min_tot_team
                        team_sizes[min_tot_team] += 1
                        team_sizes[old_team] -= 1
                        curr_first = tot_first
                        curr_third = tot_third
                        curr_score = tot_score
                    visited[curr] = True
                    
        curr_cost = score(G_copy)
        if curr_cost <= min_cost:
            min_cost = curr_cost
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]['team'] = min_G.nodes[i]['team']
    return


In [ ]:
def solve(G: nx.Graph):
    V = G.number_of_nodes()
    upper_k = 2*int(((V+1)**(1/2)))
    possible_k = range(1, upper_k)
    memo = {}
    min_score = float("inf")
    min_G = G.copy()
    
    def fake_DP(G, nodes, k):
        if nodes == []:
            return (score(G, True))
        curr = nodes[0]
        below = fake_DP(G, nodes[1:], k)
        curr_first, curr_second, curr_third = below[0], below[1], below[2]
        curr_second = K_COEFFICIENT*math.exp(0.5*k)
        best_score = sum(below)
        old_team = G.nodes[curr]["team"]
        best_team = old_team
        for new_team in range(1, k+1):
            temp_first = new_first(G, curr, old_team, new_team, curr_first)
            temp_third = new_third(team_sizes, old_team, new_team, curr_third, V, k)
            temp_score = temp_first + curr_second + temp_third
            if temp_score < best_score:
                curr_first = temp_first
                curr_third = temp_third
                best_score = temp_score
                best_team = new_team
        G.nodes[curr]["team"] = best_team
        team_sizes[old_team] -= 1
        team_sizes[best_team] += 1
        return (curr_first, curr_second, curr_third)

    for k in possible_k:
        G_copy, G_copy_score = partite(G.copy(), k)
        if G_copy_score < min_score:
            min_score = G_copy_score
            min_G = G_copy.copy()
        team_sizes = {i:0 for i in range(1, k+1)}
        for i in G_copy:
            team_sizes[G_copy.nodes[i]["team"]] += 1
        k_score = sum(fake_DP(G_copy, list(G_copy.nodes), k))
        if k_score < min_score:
            min_score = k_score
            min_G = G_copy.copy()
    for i in min_G.nodes:
        G.nodes[i]["team"] = min_G.nodes[i]["team"]
    return
